In [1]:
import json

FIRST = "o"
SECOND = "x"
BLANK = "-"

EMPTYBOARD = BLANK * 9

class GameGraph:
    # Initializes the game graph with a blank board as the root
    def __init__(self):
        self.root = BLANK * 9  # "---------", blank board
        self.nodes = {self.root: []}  # Dictionary with board states as keys and their child states as values
        self.generateTree()
        self.board_score = self.minimax() # Pre-computes the minimax values for each board state

    # Adds nodes and their adjacent nodes to the graph
    def addNode(self, root, adj):
        self.nodes[root] = adj
        return

    # Gets the adjacent nodes of a given board state
    def getAdjacentNodes(self, root):
        return self.nodes[root]

    # Generates the game tree starting from the root node
    def generateTree(self, root=None):
        if (root is None):
            root = self.root

        self.addNode(root, self.generateAdjacentNodes(root))
        for adj in self.nodes[root]:
            self.generateTree(adj)

        return

    # Generates all possible next moves of a given board state
    def generateAdjacentNodes(self, root):
        adj = []
        if isGameOver(root):
            return adj  # Empty list indicates no further possible states

        next = FIRST if getFirst(root) else SECOND
        for i in range(9):
            if (root[i] == BLANK):
                newNode = root[:i] + next + root[i + 1:]
                adj.append(newNode)
        return adj

    # Returns >0 if the first player wins, <0 if the second player wins, and 0 if it's a tie
    # Depth is used to prioritize stalling over an immediate loss
    def score(self, root, depth):
        match getWinner(root):
            case 1:
                return 10 - depth
            case -1:
                return depth - 10
            case _:
                return 0

    # Populates a dictionary with minimax score values for each board state
    def minimax(self, root=None, depth=0, maximizing=True):
        if (root is None):
            root = self.root
        if (isGameOver(root)):
            return {root: self.score(root, depth)}

        depth += 1
        board_score = {}  # Key-value pairs of game state and score

        adj_moves = self.getAdjacentNodes(root)
        for adj_node in adj_moves:
            board_score.update(self.minimax(adj_node, depth, not maximizing))

        if maximizing:
            board_score[root] = max(board_score.values())
        else:
            board_score[root] = min(board_score.values())

        return board_score

    # Prints the chosen board state and its immediate future board states
    def printNodes(self, root=None):
        if (root is None):
            root = self.root

        print("current node:\n" + nodeToStr(root) + "\n")
        print("current adjacent node(s):")
        for adj in self.getAdjacentNodes(root):
            print(nodeToStr(adj) + "\n")

# Returns a board state in |7|-|9| format
# Board goes from          |-|5|-|
# Bottom to top            |1|-|3|
def nodeToStr(root=None):
    if (root is None):
        root = "123456789"

    rows = []
    # Print in reverse order because I like numpad notation
    for subStr in [root[6:], root[3:6], root[:3]]:
        rows.append("|" + "|".join(subStr) + "|")
    return "\n".join(rows)

# Returns true if either a winner exists or all positions are filled, false otherwise
def isGameOver(root):
    # Checks to see if the board is filled
    if (BLANK not in root):
        return True

    # Checks to see if the board has three in a row
    for line in [[6, 7, 8], [3, 4, 5], [0, 1, 2], [0, 3, 6], [1, 4, 7], [2, 5, 8], [0, 4, 8], [2, 4, 6]]:
        grid0 = root[line[0]]
        grid1 = root[line[1]]
        grid2 = root[line[2]]

        if (grid0 != BLANK):
            if (grid0 == grid1 and grid1 == grid2):
                return True
    return False

# Returns 1 if the first player wins, -1 if the second player wins, and 0 if it's a tie
def getWinner(root):
    for line in [[6, 7, 8], [3, 4, 5], [0, 1, 2], [0, 3, 6], [1, 4, 7], [2, 5, 8], [0, 4, 8], [2, 4, 6]]:
        grid0 = root[line[0]]
        grid1 = root[line[1]]
        grid2 = root[line[2]]

        if (grid0 != BLANK):
            if (grid0 == grid1 and grid1 == grid2):
                return 1 if grid0 == FIRST else -1
    return 0  # On tie

class AI:
    def __init__(self, game):
        self.game = game  # GameGraph
        self.board_score = game.board_score

    # Returns the node/string representing the best next board state for AI
    def turn(self, root):
        moves = {}
        adj_nodes = self.game.getAdjacentNodes(root)
        if (len(adj_nodes) == 0):
            return root
        for next in adj_nodes:
            moves[next] = self.board_score[next]

        # Pick max if AI is playing for the P1 slot, min otherwise
        first = getFirst(root)
        if (first):
            return max(moves, key=moves.get)
        return min(moves, key=moves.get)

# Returns 1 if P1 is next to make a move, 0 otherwise
def getFirst(root):
    return root.count(BLANK) % 2

class LiveGame:
    def __init__(self, game, userFirst=True):
        self.game = game  # GameGraph
        self.userFirst = userFirst
        self.board = self.game.root
        self.opponent = AI(self.game)

    def playerTurn(self):
        next = FIRST if getFirst(self.board) else SECOND
        adj_nodes = self.game.getAdjacentNodes(self.board)
        index = int(input("Input your next position: [1-9]\nYour turn: ")) - 1
        newBoard = self.board[:index] + next + self.board[index + 1:]
        if (newBoard not in adj_nodes):
            print("Invalid move")
            return False  # Error
        self.board = newBoard
        return True

    def gameLoop(self):
        print(nodeToStr())
        # AI first turn if the human is not first
        if (not self.userFirst):
            self.board = self.opponent.turn(self.board)
            print("AI turn: \n" + nodeToStr(self.board))

        gameOver = isGameOver(self.board)
        while (not gameOver):
            isValid = self.playerTurn()
            while (not isValid):
                isValid = self.playerTurn()
            print(nodeToStr(self.board))
            gameOver = isGameOver(self.board)
            if (gameOver):
                break  # This will never be used because the AI cannot lose

            self.board = self.opponent.turn(self.board)
            print("AI turn: \n" + nodeToStr(self.board))
            gameOver = isGameOver(self.board)

        winner = getWinner(self.board)
        result = "Nobody WINS"
        match winner:
            case 1: result = f"Player {FIRST} WINS"
            case -1: result = f"Player {SECOND} WINS"

        print(result)
        print("Final board: ")
        print(nodeToStr(self.board))


# Save the game graph to a JSON file
board = GameGraph()
with open('output.json', 'w+') as f:
    json.dump(board.nodes, f, indent="\t")

# Start a live game with user input
userMark = input("Pick a side: \n" +
                 "Input \'" + FIRST + "\' to go first\n" +
                 "Input \'" + SECOND + "\' to go second\n")
game = LiveGame(board, userMark == FIRST)
game.gameLoop()


Pick a side: 
Input 'o' to go first
Input 'x' to go second
x
|7|8|9|
|4|5|6|
|1|2|3|
AI turn: 
|-|-|-|
|-|-|-|
|o|-|-|
Input your next position: [1-9]
Your turn: 5
|-|-|-|
|-|x|-|
|o|-|-|
AI turn: 
|-|-|-|
|-|x|-|
|o|o|-|
Input your next position: [1-9]
Your turn: 9
|-|-|x|
|-|x|-|
|o|o|-|
AI turn: 
|-|-|x|
|-|x|-|
|o|o|o|
Player o WINS
Final board: 
|-|-|x|
|-|x|-|
|o|o|o|
